# 检测和缓解模型中的不公平性

机器学习模型可能包含无意的偏见，这会导致公平性问题。例如，预测患糖尿病可能性的模型可能对某些年龄段效果很好，而对其他年龄段效果不佳 - 让一部分患者接受不必要的测试，或者使其不能接受可能确诊糖尿病的测试。

在此笔记本中，你将使用 **Fairlearn** 包来分析模型，并探索基于年龄的不同患者子集的预测性能差异。

> **备注**：与 Fairlearn 包的集成目前以预览版提供。你可能会遇到一些意外错误。

## 重要信息 - 公平性考虑因素

> 此笔记本旨在用作模拟练习，帮助你探索 Fairlearn 包及其与 Azure 机器学习的集成。但是，在使用工具之前，组织或数据科学团队必须探讨很多与公平性相关的考虑因素。公平性是一项复杂的社会技术挑战，不仅仅是简单地运行一个工具来分析模型。
>
> Microsoft Research 共同开发了一个[公平性清单](https://www.microsoft.com/en-us/research/publication/co-designing-checklists-to-understand-organizational-challenges-and-opportunities-around-fairness-in-ai/)，该清单为编写一行代码之前需要进行的重要讨论提供了一个很好的起点。

## 安装所需的 SDK

若要结合使用 Fairlearn 包和 Azure 机器学习，需要 Azure 机器学习和 Fairlearn Python 包，因此运行以下单元格以验证是否安装了 **azureml-contrib-fairness** 包。 

In [ ]:
!pip show azureml-contrib-fairness

还需要 **fairlearn** 包本身和 **raiwidgets** 包（Fairlearn 使用它来可视化仪表板）。运行以下单元格来安装它们。

In [ ]:
!pip install --upgrade fairlearn==0.7.0 raiwidgets

## 训练模型

首先训练分类模型，以预测患糖尿病的可能性。除了将数据分为包含特征和标签的训练集和测试集之外，还将提取敏感特征，用于定义要比较公平性的数据子集。在本例中，使用“**年龄**”列来定义两类患者：50 岁以上的患者和 50 岁及以下的患者。

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

# load the diabetes dataset
print("Loading Data...")
data = pd.read_csv('data/diabetes.csv')

# Separate features and labels
features = ['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']
X, y = data[features].values, data['Diabetic'].values

# Get sensitive features
S = data[['Age']].astype(int)
# Change value to represent age groups
S['Age'] = np.where(S.Age > 50, 'Over 50', '50 or younger')

# Split data into training set and test set
X_train, X_test, y_train, y_test, S_train, S_test = train_test_split(X, y, S, test_size=0.20, random_state=0, stratify=y)

# Train a classification model
print("Training model...")
diabetes_model = DecisionTreeClassifier().fit(X_train, y_train)

print("Model trained.")

模型训练完毕后，就可以使用 Fairlearn 包来比较不同敏感特征值的行为。在本例中，你将：

- 使用 fairlearn **selection_rate** 函数返回总人数的选择率（阳性预测结果百分比）。
- 使用 **scikit-learn** 指标函数来计算总体“准确度”、“召回率”和“精确度”指标。
- 使用 **MetricFrame** 计算“年龄”敏感特征中每个年龄段的“选择率”、“**准确度**”、“召回率”和“精确度”。请注意，你结合使用了 **fairlearn** 和 **scikit-learn** 指标函数来计算性能值。

In [ ]:
from fairlearn.metrics import selection_rate, MetricFrame
from sklearn.metrics import accuracy_score, recall_score, precision_score

# Get predictions for the witheld test data
y_hat = diabetes_model.predict(X_test)

# Get overall metrics
print("Overall Metrics:")
# Get selection rate from fairlearn
overall_selection_rate = selection_rate(y_test, y_hat) # Get selection rate from fairlearn
print("\tSelection Rate:", overall_selection_rate)
# Get standard metrics from scikit-learn
overall_accuracy = accuracy_score(y_test, y_hat)
print("\tAccuracy:", overall_accuracy)
overall_recall = recall_score(y_test, y_hat)
print("\tRecall:", overall_recall)
overall_precision = precision_score(y_test, y_hat)
print("\tPrecision:", overall_precision)

# Get metrics by sensitive group from fairlearn
print('\nMetrics by Group:')
metrics = {'selection_rate': selection_rate,
           'accuracy': accuracy_score,
           'recall': recall_score,
           'precision': precision_score}

group_metrics = MetricFrame(metrics=metrics,
                             y_true=y_test,
                             y_pred=y_hat,
                             sensitive_features=S_test['Age'])

print(group_metrics.by_group)

从这些指标中，你应该能够分辨出年龄较大的患者中有较大的比例预计患有糖尿病。两个年龄段的准确度应大致相等，但仔细检查“精确度”和“召回率”表明模型对每个年龄段的预测效果存在一定差异。

在本例中，请考虑“*召回率*”。此指标指示模型正确识别的阳性病例的比例。换句话说，在所有实际患有糖尿病的患者中，模型找到了多少名患者？此模型针年龄较大的患者给出的预测结果比针对年龄较小的患者给出的更准确。

从视觉上比较指标通常更容易。为此，你将使用 Fairlearn 公平性仪表板：

1. 运行下面的单元格，从之前创建的模型生成仪表板。
2. 当系统显示小组件时，请使用“**开始**”链接开始配置可视化。
3. 选择要比较的敏感特征（在本例中，只有一个：**年龄**）。
4. 选择要比较的模型性能指标（在本例中，模型为二元分类模型，因此选项包括“*准确度*”、“*平衡准确度*”、“*精确度*”和“*召回率*”）。从“**召回率**”开始。
5. 选择要查看的公平性比较类型。从“**群体均等差异**”开始。
6. 查看仪表板可视化效果，其中显示：
    - **性能差异** - 所选性能指标与子集的比较，包括*预测值过低* （假阴性）和*预测值过高* （假阳性）。
    - **预测差异** - 每个子集阳性病例数的比较。
7. 编辑配置，以根据不同的性能和公平性指标比较预测结果。

In [ ]:
from raiwidgets import FairnessDashboard

# View this model in Fairlearn's fairness dashboard, and see the disparities which appear:
FairnessDashboard(sensitive_features=S_test,
                   y_true=y_test,
                   y_pred={"diabetes_model": diabetes_model.predict(X_test)})

结果表明，50 岁以上的患者的选择率要比 50 岁及以下的患者高得多。但是，实际上，年龄是糖尿病的真正因素，因此在高龄患者中预计会出现更多阳性病例。

如果模型性能基于“*准确度*”（也就是模型预测正确百分比），那么对于这两个子集而言，其效果似乎大致相同。但是，基于“精确度”和“召回率”指标，该模型针对 50 岁以上的患者给出的预测结果更为准确。

让我们看看如果在训练模型时排除“**年龄**”特征会发生什么情况。

In [ ]:
# Separate features and labels
ageless = features.copy()
ageless.remove('Age')
X2, y2 = data[ageless].values, data['Diabetic'].values

# Split data into training set and test set
X_train2, X_test2, y_train2, y_test2, S_train2, S_test2 = train_test_split(X2, y2, S, test_size=0.20, random_state=0, stratify=y2)

# Train a classification model
print("Training model...")
ageless_model = DecisionTreeClassifier().fit(X_train2, y_train2)
print("Model trained.")

# View this model in Fairlearn's fairness dashboard, and see the disparities which appear:
FairnessDashboard(sensitive_features=S_test2,
                   y_true=y_test2,
                   y_pred={"ageless_diabetes_model": ageless_model.predict(X_test2)})

在仪表板中探索模型。

查看“*召回率*”时，请注意差异有所减少，但总体召回率也有所降低，因为该模型现在大大低估了高龄患者的阳性病例。尽管“**年龄**”不是训练中使用的特征，但该模型在针对高龄患者和针对低龄患者的预测效果方面仍然存在差异。

在本例中，删除“**年龄**”特征会稍微降低“*召回率*”差异，但会增加“*精确度*”和“*准确度*”方面的差异。这强调了将公平性应用于机器学习模型的主要困难之一 - 必须清楚公平性在特定上下文中的含义，并为此进行优化。

## 注册模型并将仪表板数据上传到工作区

你已经训练了模型并在此笔记本中在本地查看了仪表板，但在 Azure 机器学习工作区中注册模型并创建试验以记录仪表板数据可能会很有用，让你可以跟踪和共享公平性分析。

让我们首先注册原始模型（其中包括“**年龄**”作为特征）。

> **备注**：如果尚未与 Azure 订阅建立经过身份验证的会话，则系统将提示你通过执行以下操作进行身份验证：单击链接，输入验证码，然后登录到 Azure。

In [ ]:
from azureml.core import Workspace, Experiment, Model
import joblib
import os

# Load the Azure ML workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

# Save the trained model
model_file = 'diabetes_model.pkl'
joblib.dump(value=diabetes_model, filename=model_file)

# Register the model
print('Registering model...')
registered_model = Model.register(model_path=model_file,
                                  model_name='diabetes_classifier',
                                  workspace=ws)
model_id= registered_model.id


print('Model registered.', model_id)

现在，可以使用 FairLearn 包为一个或多个模型创建二元分类年龄段指标集，并使用 Azure 机器学习试验上传指标。

> **备注**：可能需要一点时间，并产生一些警告消息（可以忽略）。试验完成后，将下载并显示仪表板数据以验证指标是否已成功上传。

In [ ]:
from fairlearn.metrics._group_metric_set import _create_group_metric_set
from azureml.contrib.fairness import upload_dashboard_dictionary, download_dashboard_by_upload_id

#  Create a dictionary of model(s) you want to assess for fairness 
sf = { 'Age': S_test.Age}
ys_pred = { model_id:diabetes_model.predict(X_test) }
dash_dict = _create_group_metric_set(y_true=y_test,
                                    predictions=ys_pred,
                                    sensitive_features=sf,
                                    prediction_type='binary_classification')

exp = Experiment(ws, 'mslearn-diabetes-fairness')
print(exp)

run = exp.start_logging()

# Upload the dashboard to Azure Machine Learning
try:
    dashboard_title = "Fairness insights of Diabetes Classifier"
    upload_id = upload_dashboard_dictionary(run,
                                            dash_dict,
                                            dashboard_name=dashboard_title)
    print("\nUploaded to id: {0}\n".format(upload_id))

    # To test the dashboard, you can download it
    downloaded_dict = download_dashboard_by_upload_id(run, upload_id)
    print(downloaded_dict)
finally:
    run.complete()

前面的代码下载了试验中生成的指标，以确认上传已成功完成。将指标上传到试验的真正好处是，现在可以在 Azure 机器学习工作室中查看 FairLearn 仪表板。

运行下面的单元格以查看试验详细信息，然后单击小组件中的“**查看运行详细信息**”链接，以在 Azure 机器学习工作室中查看运行。然后查看试验运行的“**公平性**”选项卡，以查看分配给上传指标的公平 ID 的仪表板，该仪表板的行为与之前在此笔记本中查看的小组件的行为相同。

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

也可以通过在 Azure 机器学习工作室的“**模型**”页面中选择一个模型并查看其“**公平性**”选项卡来查找公平性仪表板。这使组织可以维护你训练和注册的模型的公平性分析日志。

## 缓解模型中的不公平性

现在，你已经分析了模型的公平性，可以使用 FairLearn 包支持的任何缓解技术来查找平衡预测性能和公平性的模型。

在本练习中，你将使用 **GridSearch** 功能，该功能可训练多个模型，以尽量减少数据集中敏感特征（在本例中为年龄段）的预测性能差异。你将通过应用 **EqualizedOdds** 奇偶校验约束来优化模型，该约束将尝试确保每个敏感特征分组的模型显示出相似的真阳性率和假阳性率。 

> *这可能需要一些时间才能运行*

In [ ]:
from fairlearn.reductions import GridSearch, EqualizedOdds
import joblib
import os

print('Finding mitigated models...')

# Train multiple models
sweep = GridSearch(DecisionTreeClassifier(),
                   constraints=EqualizedOdds(),
                   grid_size=20)

sweep.fit(X_train, y_train, sensitive_features=S_train.Age)
models = sweep.predictors_

# Save the models and get predictions from them (plus the original unmitigated one for comparison)
model_dir = 'mitigated_models'
os.makedirs(model_dir, exist_ok=True)
model_name = 'diabetes_unmitigated'
print(model_name)
joblib.dump(value=diabetes_model, filename=os.path.join(model_dir, '{0}.pkl'.format(model_name)))
predictions = {model_name: diabetes_model.predict(X_test)}
i = 0
for model in models:
    i += 1
    model_name = 'diabetes_mitigated_{0}'.format(i)
    print(model_name)
    joblib.dump(value=model, filename=os.path.join(model_dir, '{0}.pkl'.format(model_name)))
    predictions[model_name] = model.predict(X_test)


现在，可以使用 FairLearn 仪表板来比较已缓解的模型：

运行以下单元格，然后使用向导按“**召回率**”可视化“**年龄**”。

In [ ]:
FairnessDashboard(sensitive_features=S_test,
                   y_true=y_test,
                   y_pred=predictions)

这些模型显示在散点图上。可以通过衡量预测结果中的差异（也就是选择率）或所选性能指标中的差异（在本例中为*召回率*）来比较模型。在本场景中，我们预计选择率存在差异（因为我们知道年龄是糖尿病的一个因素，年龄较大的人群中的阳性病例更多）。我们感兴趣的是预测性能方面的差异，因此选择用于衡量“**召回率中的差异**”的选项。

图表在 X 轴上显示了具有总体“召回率”指标的模型群集，而在 Y 轴上显示了召回*率差异*。因此，理想模型（召回率高且差异小）将位于图的右下角。可以选择适当平衡预测性能和公平性，以满足特定需求，然后选择合适的模型以查看其详细信息。

需要强调的一个重点是，对模型应用公平性缓解措施是在整体预测性能与敏感特征组之间的差异之间的权衡 - 通常，必须牺牲一些整体预测性能，以确保模型能够针对所有年龄段人群进行公平预测。

> **备注**：查看“*精确度*”指标可能会生成一条警告，即由于没有预测样本而将“精确度”设置为 0.0，你可以忽略此警告。

## 将缓解仪表板指标上传到 Azure 机器学习

照旧，建议跟踪缓解试验。为此，可以：

1. 注册由 GridSearch 进程找到的模型。
2. 计算模型的性能指标和差异指标。
3. 在 Azure 机器学习试验中上传指标。

In [ ]:
# Register the models
registered_model_predictions = dict()
for model_name, prediction_data in predictions.items():
    model_file = os.path.join(model_dir, model_name + ".pkl")
    registered_model = Model.register(model_path=model_file,
                                      model_name=model_name,
                                      workspace=ws)
    registered_model_predictions[registered_model.id] = prediction_data

#  Create a group metric set for binary classification based on the Age feature for all of the models
sf = { 'Age': S_test.Age}
dash_dict = _create_group_metric_set(y_true=y_test,
                                     predictions=registered_model_predictions,
                                     sensitive_features=sf,
                                     prediction_type='binary_classification')

exp = Experiment(ws, "mslearn-diabetes-fairness")
print(exp)

run = exp.start_logging()
RunDetails(run).show()

# Upload the dashboard to Azure Machine Learning
try:
    dashboard_title = "Fairness Comparison of Diabetes Models"
    upload_id = upload_dashboard_dictionary(run,
                                            dash_dict,
                                            dashboard_name=dashboard_title)
    print("\nUploaded to id: {0}\n".format(upload_id))
finally:
    run.complete()

> **备注**：可能会显示一条由于没有预测样本而将“精确度”设置为 0.0 的警告，你可以忽略此警告。


试验完成运行后，单击小组件中的“**查看运行详细信息**”链接以在 Azure 机器学习工作室中查看运行（可能需要滚动到初始输出以查看小组件），并在“**公平性**”选项卡上查看 FairLearn 仪表板。